In [1]:
#Dependencies
import requests
import json
import pandas as pd
import numpy as np
from apikey import api_key

In [2]:
#Create base urls
baseurl = "https://api.census.gov/data/"

#Create suffix for url
suffix = "&for=county:095&in=state:12&key=" + api_key

In [6]:
#Import csv
path = "../Resources/api_pulls.csv"
file = pd.read_csv(path)

#splits the column names by the "_" delimiter to sort year easier, makes array of first word in the split
frontsplit = file.columns.str.split('_').str[0]
years = []
#loop to take the year values out of split array
for element in frontsplit:
    if element != 'Formal':
        if element != 'Label':
            years.append(element)
#printing array to show years being looked at
print(years)


labels = []
for label in file["Label_Name"]:
    labels.append(label)
labels.append("State")
labels.append("County")
print(labels)

['2017', '2016', '2015', '2014']
['Households', 'Households (Family)', 'Households (Non-Family)', 'Average Household Size', 'Average Family Size', 'Number of Higher Ed Enrolled', 'Education of Some College - No Degree Attained', "Education of Associate's Degree Attained", "Education of Bachelor's Degree Attained", 'Education of Graduate or Professional Degree Attained', 'Median Household Income (in dollars)', 'Mean Household Income (in dollars)', 'Median Family Income (in dollars)', 'Mean Family Income (in dollars)', 'Median Non-Family Income (in dollars)', 'Mean Non-Family Income (in dollars)', 'Age of 20 to 24 years', 'Age of 25 to 34 years', 'Age of 35 to 44 years', 'Age of 45 to 54 years', 'Age of 55 to 59 years', 'Age of 60 to 64 years', 'Age of 65 to 74 years', 'Age of 75 to 84 years', 'Age of 85 years and over', 'Race - White', 'Race - Black or African American', 'Race - American Indian and Alaska Native', 'Race - Asian', 'Race - Native Hawaiian and Other Pacific Islander', 'Rac

In [7]:
#Begin For-Loops & Pulling APIs
response = {}
counter = 0
for year in years:
    url = baseurl + year + "/acs/acs1/profile?get="
    
    #building path to pull api with proper csv column
    filename = year + "_Variable"
    
    for row in file[filename]:
        url = url + row + ","
    
    #deletes trailing comma from the above loop
    url = url[:-1]
    url = url + suffix    
    
    response[counter] = requests.get(url).json()
    counter += 1

In [10]:
#initialize dictionary to hold response data
data = {}
counter = 0
#loop to organize data into dictionary
for year in years:
    data[year] = response[counter][1]
    counter += 1

#put data into dataframe    
df = pd.DataFrame.from_dict(data, orient='index', columns= labels)
df = df.drop(['State', 'County'], axis=1)
df

,Households,Households (Family),Households (Non-Family),Average Household Size,Average Family Size,Number of Higher Ed Enrolled,Education of Some College - No Degree Attained,Education of Associate's Degree Attained,Education of Bachelor's Degree Attained,Education of Graduate or Professional Degree Attained,...,Race - Black or African American,Race - American Indian and Alaska Native,Race - Asian,Race - Native Hawaiian and Other Pacific Islander,Race - Other,Race - White and Black or African American,Race - White and American Indian and Alaska Native,Race - White and Asian,Race - Black or African American and American Indian and Alaska Native,Race - Hispanic or Latino
2017,460602,303666,156936,2.86,3.47,117462,169812,114043,206650,101612,...,286324,2743,69335,1623,107587,15643,4357,10259,633,1348975
2016,468515,301665,166850,2.74,3.34,130436,161389,90715,196945,97948,...,276302,1697,70844,50,67741,12551,4753,6637,1229,1314367
2015,457736,301978,155758,2.74,3.30,113112,172427,89449,185472,88758,...,263369,2783,68409,1617,85157,11182,3874,8804,1306,1288126
2014,444543,283682,160861,2.75,3.40,122968,161727,95527,170560,88350,...,260845,2682,64590,623,66597,10523,3694,7878,660,1253001


In [12]:
#csv output
outputfile = "../resources/demographics.csv"
df.to_csv(outputfile)